In [ ]:
# | default_exp _components.producer_decorator

In [ ]:
# | export

import random
import asyncio
import functools
import json
from asyncio import iscoroutinefunction  # do not use the version from inspect
from collections import namedtuple
from dataclasses import dataclass
from typing import *

import nest_asyncio
from aiokafka import AIOKafkaProducer
from pydantic import BaseModel

from fastkafka._components.meta import export

In [ ]:
import asyncio
from contextlib import asynccontextmanager
import unittest
from unittest.mock import Mock
from itertools import product

from pydantic import Field

from fastkafka._testing.apache_kafka_broker import ApacheKafkaBroker
from fastkafka._testing.test_utils import mock_AIOKafkaProducer_send
from fastkafka.encoder import avro_encoder, json_encoder

In [ ]:
# | export


BaseSubmodel = TypeVar("BaseSubmodel", bound=BaseModel)
BaseSubmodel


@dataclass
@export("fastkafka")
class KafkaEvent(Generic[BaseSubmodel]):
    """
    A generic class for representing Kafka events. Based on BaseSubmodel, bound to pydantic.BaseModel

    Attributes:
        message (BaseSubmodel): The message contained in the Kafka event, can be of type pydantic.BaseModel.
        key (bytes, optional): The optional key used to identify the Kafka event.
    """

    message: BaseSubmodel
    key: Optional[bytes] = None

In [ ]:
event = KafkaEvent("Some message")
assert event.message == "Some message"
assert event.key == None

event = KafkaEvent("Some message", b"123")
assert event.message == "Some message"
assert event.key == b"123"

In [ ]:
# | export

ProduceReturnTypes = Union[
    BaseModel, KafkaEvent[BaseModel], List[BaseModel], KafkaEvent[List[BaseModel]]
]

ProduceCallable = Union[
    Callable[..., ProduceReturnTypes], Callable[..., Awaitable[ProduceReturnTypes]]
]

In [ ]:
# # | export


# def _to_json_utf8(o: Any) -> bytes:
#     """Converts to JSON and then encodes with UTF-8"""
#     if hasattr(o, "json"):
#         return o.json().encode("utf-8")  # type: ignore
#     else:
#         return json.dumps(o).encode("utf-8")

In [ ]:
# assert _to_json_utf8({"a": 1, "b": [2, 3]}) == b'{"a": 1, "b": [2, 3]}'


class A(BaseModel):
    name: str = Field()
    age: int


# assert _to_json_utf8(A(name="Davor", age=12)) == b'{"name": "Davor", "age": 12}'

In [ ]:
# | export


def _wrap_in_event(message: Union[BaseModel, KafkaEvent]) -> KafkaEvent:
    return message if type(message) == KafkaEvent else KafkaEvent(message)

In [ ]:
message = A(name="Davor", age=12)
wrapped = _wrap_in_event(message)

assert type(wrapped) == KafkaEvent
assert wrapped.message == message
assert wrapped.key == None

In [ ]:
message = KafkaEvent(A(name="Davor", age=12), b"123")
wrapped = _wrap_in_event(message)

assert type(wrapped) == KafkaEvent
assert wrapped.message == message.message
assert wrapped.key == b"123"

In [ ]:
# | export


def get_loop() -> asyncio.AbstractEventLoop:
    try:
        loop: asyncio.AbstractEventLoop = asyncio.get_event_loop()
    except RuntimeError as e:
        loop = asyncio.new_event_loop()

    if loop.is_running():
        nest_asyncio.apply(loop)

    return loop

In [ ]:
loop = get_loop()

assert isinstance(loop, asyncio.AbstractEventLoop)

In [ ]:
# | export


def release_callback(fut: asyncio.Future) -> None:
    pass

In [ ]:
# | export

async def produce_single(
    producer: AIOKafkaProducer,
    topic: str,
    encoder_fn: Callable[[BaseModel], bytes],
    wrapped_val: KafkaEvent[BaseModel],
) -> ProduceReturnTypes:
    fut = await producer.send(
        topic, encoder_fn(wrapped_val.message), key=wrapped_val.key
    )
    fut.add_done_callback(release_callback)

In [ ]:
with ApacheKafkaBroker(topics=["test_topic"], apply_nest_asyncio=True) as broker:
    producer = AIOKafkaProducer(bootstrap_servers=broker)
    await producer.start()

    await produce_single(
        producer,
        topic="test_topic",
        encoder_fn=json_encoder,
        wrapped_val=KafkaEvent(message=A(name="Davor", age=12), key=b"test"),
    )
    
    await producer.stop()

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.sta

In [ ]:
# | export


async def send_batch(producer, topic, batch, key):
    partitions = await producer.partitions_for(topic)
    if key == None:
        partition = random.choice(tuple(partitions))
    else:
        partition = producer._partition(topic, None, None, None, key, None)
    await producer.send_batch(batch, topic, partition=partition)


async def produce_batch(
    producer: AIOKafkaProducer,
    topic: str,
    encoder_fn: Callable[[BaseModel], bytes],
    wrapped_val: KafkaEvent[List[BaseModel]],
) -> ProduceReturnTypes:
    batch = producer.create_batch()

    for message in wrapped_val.message:
        metadata = batch.append(
            key=wrapped_val.key, value=encoder_fn(message), timestamp=None
        )
        if metadata == None:
            # send batch
            await send_batch(producer, topic, batch, wrapped_val.key)
            # create new batch
            batch = producer.create_batch()
            batch.append(key=None, value=encoder_fn(message), timestamp=None)
    
    await send_batch(producer, topic, batch, wrapped_val.key)

In [ ]:
msgs = [A(name="Davor", age=12) for _ in range(500)]

with ApacheKafkaBroker(topics=["test_topic"], apply_nest_asyncio=True) as broker:
    producer = AIOKafkaProducer(bootstrap_servers=broker)
    await producer.start()

    await produce_batch(
        producer,
        topic="test_topic",
        encoder_fn=json_encoder,
        wrapped_val=KafkaEvent(message=msgs, key=b"test"),
    )
    
    await producer.stop()

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): exited.
[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBro

In [ ]:
# | export


def producer_decorator(
    producer_store: Dict[str, Any],
    func: ProduceCallable,
    topic: str,
    encoder_fn: Callable[[BaseModel], bytes],
) -> ProduceCallable:
    """todo: write documentation"""

    loop = get_loop()

    @functools.wraps(func)
    async def _produce_async(
        *args: List[Any],
        topic: str = topic,
        encoder_fn: Callable[[BaseModel], bytes] = encoder_fn,
        producer_store: Dict[str, Any] = producer_store,
        f: Callable[..., Awaitable[ProduceReturnTypes]] = func,  # type: ignore
        **kwargs: Any
    ) -> ProduceReturnTypes:
        return_val = await f(*args, **kwargs)
        wrapped_val = _wrap_in_event(return_val)
        _, producer, _ = producer_store[topic]

        if isinstance(wrapped_val.message, list):
            await produce_batch(producer, topic, encoder_fn, wrapped_val)
        else:
            await produce_single(producer, topic, encoder_fn, wrapped_val)
        return return_val

    @functools.wraps(func)
    def _produce_sync(
        *args: List[Any],
        topic: str = topic,
        encoder_fn: Callable[[BaseModel], bytes] = encoder_fn,
        producer_store: Dict[str, Any] = producer_store,
        f: Callable[..., ProduceReturnTypes] = func,  # type: ignore
        loop: asyncio.AbstractEventLoop = loop,
        **kwargs: Any
    ) -> ProduceReturnTypes:
        return_val = f(*args, **kwargs)
        wrapped_val = _wrap_in_event(return_val)
        _, producer, _ = producer_store[topic]
        if isinstance(wrapped_val.message, list):
            loop.run_until_complete(
                produce_batch(producer, topic, encoder_fn, wrapped_val)
            )
        else:
            loop.run_until_complete(
                produce_single(producer, topic, encoder_fn, wrapped_val)
            )
        return return_val

    return _produce_async if iscoroutinefunction(func) else _produce_sync

In [ ]:
class MockMsg(BaseModel):
    name: str = "Micky Mouse"
    id: int = 123


mock_msg = MockMsg()

topic = "test_topic"

In [ ]:
@asynccontextmanager
async def mock_producer_env(
    is_sync: bool,
) -> AsyncGenerator[Tuple[Mock, AIOKafkaProducer], None]:
    try:
        with mock_AIOKafkaProducer_send() as send_mock, unittest.mock.patch(
            "__main__.AIOKafkaProducer.send_batch"
        ) as send_batch_mock, unittest.mock.patch(
            "__main__.AIOKafkaProducer.create_batch"
        ) as create_batch_mock:
            batch_mock = Mock()
            create_batch_mock.return_value = batch_mock
            async with ApacheKafkaBroker(topics=[topic]) as bootstrap_server:
                producer = AIOKafkaProducer(bootstrap_servers=bootstrap_server)
                await producer.start()
                yield send_mock, batch_mock, send_batch_mock, producer
    finally:
        await producer.stop()

In [ ]:
async def func_async(mock_msg: MockMsg) -> MockMsg:
    return mock_msg


def func_sync(mock_msg: MockMsg) -> MockMsg:
    return mock_msg


for is_sync, encoder_fn in product([True, False], [json_encoder, avro_encoder]):
    print(f"Testing with: {is_sync=} , {encoder_fn=}")
    async with mock_producer_env(is_sync=is_sync) as (send_mock, _, _, producer):
        test_func = producer_decorator(
            {topic: (None, producer, None)},
            func_sync if is_sync else func_async,
            topic,
            encoder_fn=encoder_fn,
        )

        assert iscoroutinefunction(test_func) != is_sync

        value = test_func(mock_msg) if is_sync else await test_func(mock_msg)

        send_mock.assert_called_once_with(topic, encoder_fn(mock_msg), key=None)

        assert value == mock_msg

Testing with: is_sync=True , encoder_fn=<function json_encoder>
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 47692...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 47692 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 47313...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 47313 terminated.
Testing with: is_sync=True , encoder_fn=<function avro_encoder>
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._c

In [ ]:
test_key = b"key"

async def func_async(mock_msg: MockMsg) -> KafkaEvent[MockMsg]:
    return KafkaEvent(mock_msg, test_key)


def func_sync(mock_msg: MockMsg) -> KafkaEvent[MockMsg]:
    return KafkaEvent(mock_msg, test_key)


for is_sync, encoder_fn in product([True, False], [json_encoder, avro_encoder]):
    print(f"Testing with: {is_sync=} , {encoder_fn=}")
    async with mock_producer_env(is_sync=is_sync) as (send_mock, _, _, producer):
        test_func = producer_decorator(
            {topic: (None, producer, None)},
            func_sync if is_sync else func_async,
            topic,
            encoder_fn=encoder_fn,
        )

        assert iscoroutinefunction(test_func) != is_sync

        value = test_func(mock_msg) if is_sync else await test_func(mock_msg)

        send_mock.assert_called_once_with(topic, encoder_fn(mock_msg), key=test_key)

        assert value == KafkaEvent(mock_msg, test_key)

Testing with: is_sync=True , encoder_fn=<function json_encoder>
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 40916...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 40916 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 40537...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 40537 terminated.
Testing with: is_sync=True , encoder_fn=<function avro_encoder>
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._c

In [ ]:
batch_size = 123

async def func_async(mock_msg: MockMsg) -> List[MockMsg]:
    return [mock_msg]*batch_size


def func_sync(mock_msg: MockMsg) -> List[MockMsg]:
    return [mock_msg]*batch_size


for is_sync, encoder_fn in product([True, False], [json_encoder, avro_encoder]):
    print(f"Testing with: {is_sync=} , {encoder_fn=}")
    async with mock_producer_env(is_sync=is_sync) as (_, batch_mock, send_batch_mock, producer):
        test_func = producer_decorator(
            {topic: (None, producer, None)},
            func_sync if is_sync else func_async,
            topic,
            encoder_fn=encoder_fn,
        )

        assert iscoroutinefunction(test_func) != is_sync

        value = test_func(mock_msg) if is_sync else await test_func(mock_msg)

        send_batch_mock.assert_called_once_with(batch_mock, topic, partition=0)

        assert value == [mock_msg]*batch_size

Testing with: is_sync=True , encoder_fn=<function json_encoder>
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 58524...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 58524 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 58145...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 58145 terminated.
Testing with: is_sync=True , encoder_fn=<function avro_encoder>
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._c